# 2019-2020 NBA SEASON ANALYSIS

#### LOGISTIC REGRESSION - is team going to make a playoff based on type of players in a team
#### CORRELATIONS - how important is specific players for a success in playoffs

We're analysing NBA 2019-2020 season. 
With the increasing role of 3 point shooting many specialists claim that in today's basketball you need new kind of players.
One of the most valuable players right now are big guys (SF, PF, C) who can drible, shot 3's and pass the ball to his teamates. 
I will calculate if this is true and try to predict if team will get to the playoffs or not.

In [135]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

## Loading the data

In [136]:
players = pd.read_csv('players.csv', encoding='latin-1')
raw_teams = pd.read_csv('teams.csv')

In [137]:
pd.set_option('display.max_columns', None)

## Exploratory Data Analysis

In [138]:
print(players.info())
players.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      651 non-null    int64  
 1   Player  651 non-null    object 
 2   Pos     651 non-null    object 
 3   Age     651 non-null    int64  
 4   Height  651 non-null    int64  
 5   Weight  651 non-null    int64  
 6   Tm      651 non-null    object 
 7   G       651 non-null    int64  
 8   GS      651 non-null    int64  
 9   MP      651 non-null    float64
 10  FG      651 non-null    float64
 11  FGA     651 non-null    float64
 12  FG%     649 non-null    float64
 13  3P      651 non-null    float64
 14  3PA     651 non-null    float64
 15  3P%     616 non-null    float64
 16  2P      651 non-null    float64
 17  2PA     651 non-null    float64
 18  2P%     645 non-null    float64
 19  eFG%    649 non-null    float64
 20  FT      651 non-null    float64
 21  FTA     651 non-null    float64
 22  FT

,Rk,Player,Pos,Age,Height,Weight,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
270,209,Jaxson Hayes,C,19,211,100,NOP,64,14,16.9,2.7,4.0,0.672,0.0,0.1,0.250,2.7,4.0,0.678,0.674,1.9,3.0,0.647,1.5,2.5,4.0,0.9,0.4,0.9,0.8,2.5,7.4
183,140,Andre Drummond,C,26,208,122,TOT,57,56,33.0,7.3,13.8,0.533,0.1,0.6,0.143,7.2,13.1,0.551,0.536,3.0,5.2,0.575,4.4,10.8,15.2,2.7,1.9,1.6,3.6,3.5,17.7
267,206,Josh Hart,SF,24,196,98,NOP,65,16,27.0,3.5,8.2,0.423,1.9,5.4,0.342,1.6,2.7,0.584,0.537,1.3,1.8,0.739,0.9,5.6,6.5,1.7,1.0,0.4,1.2,2.5,10.1


In [139]:
print(raw_teams.info())
raw_teams.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rk             30 non-null     int64  
 1   Team           30 non-null     object 
 2   Short_team     30 non-null     object 
 3   G              30 non-null     int64  
 4   MP             30 non-null     int64  
 5   FG             30 non-null     float64
 6   FGA            30 non-null     float64
 7   FG%            30 non-null     float64
 8   3P             30 non-null     float64
 9   3PA            30 non-null     float64
 10  3P%            30 non-null     float64
 11  2P             30 non-null     float64
 12  2PA            30 non-null     float64
 13  2P%            30 non-null     float64
 14  FT             30 non-null     float64
 15  FTA            30 non-null     float64
 16  FT%            30 non-null     float64
 17  ORB            30 non-null     float64
 18  DRB         

,Rk,Team,Short_team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,OFS/100 POSS,DEF/100 POSS,EAST/WEST,POSITION,PLAYOFF,PLAYOFF_SCORE
21,22,Orlando Magic,ORL,73,17570,39.8,89.6,0.444,11.2,32.6,0.343,28.6,57.0,0.502,17.8,22.9,0.774,10.4,34.6,45.0,24.2,8.3,5.5,13.0,18.5,108.5,109.5,EAST,8,yes,first round
4,5,Los Angeles Clippers,LAC,72,17380,40.7,87.4,0.466,12.2,32.8,0.371,28.5,54.6,0.522,20.4,25.8,0.791,10.4,36.2,46.7,23.2,6.9,4.6,14.3,21.7,113.9,107.6,WEST,2,yes,quarter-finals
28,29,Golden State Warriors,GSW,65,15725,38.2,87.2,0.438,10.3,30.9,0.334,27.9,56.3,0.495,18.5,23.0,0.803,9.8,32.5,42.4,25.3,8.1,4.5,14.7,19.8,105.2,113.8,WEST,15,no,off-playoffs


## Filtering datasets

In [140]:
players = players.drop(['Rk','DRB', 'PF'], axis = 1)

In [141]:
players.sample(5)

,Player,Pos,Age,Height,Weight,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS
322,Ty Jerome,SG,22,196,88,PHO,31,0,10.6,1.2,3.6,0.336,0.5,1.6,0.280,0.8,2.0,0.381,0.398,0.4,0.5,0.750,0.2,1.5,1.4,0.5,0.1,0.6,3.3
29,RJ Barrett,SG,19,198,97,NYK,56,55,30.4,5.2,13.0,0.402,1.1,3.5,0.320,4.1,9.5,0.432,0.445,2.8,4.5,0.614,0.9,5.0,2.6,1.0,0.3,2.2,14.3
313,Josh Jackson,SF,22,203,91,MEM,22,0,17.3,3.2,7.2,0.440,1.0,3.3,0.319,2.1,4.0,0.540,0.513,1.6,2.3,0.700,0.5,3.0,1.6,0.8,0.4,1.3,9.0
368,Courtney Lee,SG,34,196,91,DAL,24,9,14.4,1.7,3.4,0.488,0.9,2.0,0.447,0.8,1.5,0.543,0.616,0.3,0.3,0.857,0.3,1.3,0.5,0.8,0.3,0.5,4.5
308,Kyrie Irving,PG,27,191,87,BRK,20,20,32.9,10.0,20.8,0.478,2.8,7.1,0.394,7.2,13.7,0.522,0.546,4.7,5.1,0.922,1.1,5.2,6.4,1.4,0.5,2.6,27.4


In [142]:
raw_teams.sample(5)

,Rk,Team,Short_team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,OFS/100 POSS,DEF/100 POSS,EAST/WEST,POSITION,PLAYOFF,PLAYOFF_SCORE
5,6,New Orleans Pelicans,NOP,72,17430,40.7,87.6,0.465,13.0,35.3,0.370,27.6,52.3,0.528,16.3,22.4,0.729,10.6,33.9,44.4,25.6,7.2,4.8,15.7,20.2,110.7,111.9,WEST,13,no,off-playoffs
20,21,Sacramento Kings,SAC,72,17455,40.9,88.4,0.462,12.7,34.9,0.364,28.2,53.5,0.527,15.7,20.3,0.770,9.7,33.0,42.6,23.8,7.7,4.1,14.5,22.2,110.2,112.2,WEST,12,no,off-playoffs
11,12,Denver Nuggets,DEN,73,17745,42.7,90.4,0.473,11.2,31.1,0.359,31.6,59.2,0.533,16.5,21.3,0.777,10.9,33.9,44.9,27.2,8.2,4.7,14.0,20.7,113.1,111.0,WEST,3,yes,semi-finals
7,8,Washington Wizards,WAS,72,17355,40.3,88.1,0.457,11.6,31.6,0.368,28.6,56.6,0.506,18.8,23.8,0.788,9.9,30.9,40.8,24.3,7.7,4.2,13.7,22.0,110.9,115.5,EAST,10,no,off-playoffs
8,9,Memphis Grizzlies,MEM,73,17570,41.3,88.1,0.468,10.6,30.5,0.347,30.7,57.6,0.532,16.1,21.1,0.763,10.0,35.1,45.1,26.1,7.7,5.3,14.8,20.5,109.2,110.3,WEST,9,no,off-playoffs


We're droping all columns except columns with final season results.

In [143]:
teams = raw_teams.drop(['Rk', 'G','MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'FT', 'FTA', 'FT%','ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'OFS/100 POSS', 'DEF/100 POSS', 'POSITION'], axis = 1)

In [144]:
teams.sample(5)

,Team,Short_team,EAST/WEST,PLAYOFF,PLAYOFF_SCORE
29,Charlotte Hornets,CHA,WEST,no,off-playoffs
16,Brooklyn Nets,BRK,EAST,yes,first round
19,Oklahoma City Thunder,OKC,WEST,yes,first round
26,Cleveland Cavaliers,CLE,EAST,no,off-playoffs
18,Indiana Pacers,IND,EAST,yes,first round


In [145]:
players.describe(include = 'all')

,Player,Pos,Age,Height,Weight,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS
count,651,651,651.000000,651.000000,651.000000,651,651.00000,651.000000,651.000000,651.000000,651.000000,649.000000,651.000000,651.000000,616.000000,651.000000,651.000000,645.000000,649.000000,651.000000,651.000000,618.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000
unique,529,14,NaN,NaN,NaN,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Jordan McRae,SG,NaN,NaN,NaN,TOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,4,143,NaN,NaN,NaN,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,25.763441,200.161290,97.427035,NaN,38.21659,17.537634,19.179724,3.097849,6.861444,0.442891,0.938863,2.695392,0.313438,2.160829,4.167435,0.507447,0.506704,1.337788,1.750538,0.743769,0.817051,3.563287,1.843932,0.617665,0.398157,1.101075,8.461290
std,NaN,NaN,4.024840,8.381304,10.495368,NaN,22.94443,23.022124,9.060972,2.222285,4.667673,0.111526,0.849802,2.191573,0.133521,1.777765,3.284642,0.124870,0.110181,1.338949,1.658084,0.150642,0.727948,2.429509,1.698073,0.406082,0.409010,0.807703,6.197278
min,NaN,NaN,19.000000,175.000000,73.000000,NaN,1.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,23.000000,196.000000,90.000000,NaN,15.50000,0.000000,12.000000,1.400000,3.400000,0.399000,0.300000,1.000000,0.270750,0.850000,1.800000,0.459000,0.475000,0.500000,0.700000,0.667000,0.300000,1.900000,0.700000,0.300000,0.100000,0.600000,3.800000
50%,NaN,NaN,25.000000,201.000000,97.000000,NaN,41.00000,4.000000,18.500000,2.500000,5.600000,0.439000,0.800000,2.300000,0.339000,1.600000,3.100000,0.509000,0.518000,0.900000,1.300000,0.770500,0.600000,3.100000,1.300000,0.600000,0.300000,0.900000,6.800000
75%,NaN,NaN,28.000000,206.000000,104.000000,NaN,59.00000,29.500000,26.700000,4.300000,9.400000,0.486000,1.400000,3.800000,0.378250,2.900000,5.500000,0.569000,0.557000,1.700000,2.300000,0.836000,1.100000,4.650000,2.200000,0.900000,0.500000,1.400000,11.650000


## Missing values

In [146]:
players.name = 'Players'
teams.name = 'Teams'
dfs = [players, teams]
def display_missing(df):    
    for col in df.columns.tolist():          
        print('{} column missing values: {}'.format(col, df[col].isnull().sum()))
    print('\n')
    
for df in dfs:
    print('{}'.format(df.name))
    display_missing(df)

Players
Player column missing values: 0
Pos column missing values: 0
Age column missing values: 0
Height column missing values: 0
Weight column missing values: 0
Tm column missing values: 0
G column missing values: 0
GS column missing values: 0
MP column missing values: 0
FG column missing values: 0
FGA column missing values: 0
FG% column missing values: 2
3P column missing values: 0
3PA column missing values: 0
3P% column missing values: 35
2P column missing values: 0
2PA column missing values: 0
2P% column missing values: 6
eFG% column missing values: 2
FT column missing values: 0
FTA column missing values: 0
FT% column missing values: 33
ORB column missing values: 0
TRB column missing values: 0
AST column missing values: 0
STL column missing values: 0
BLK column missing values: 0
TOV column missing values: 0
PTS column missing values: 0


Teams
Team column missing values: 0
Short_team column missing values: 0
EAST/WEST column missing values: 0
PLAYOFF column missing values: 0
PLAYOF

#### FG% and 3P%

It's easy to calculate and fill. Percentage is the outcome of (FG / FGA)

In [147]:
players[players['FG%'].isnull()]

,Player,Pos,Age,Height,Weight,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS
65,Marques Bolden,C,21,208,113,CLE,1,0,3.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,2.0,0.0,1.0,0.0,0.0,0.0
386,J.P. Macura,SG,24,196,93,CLE,1,0,1.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
#Not relevant players, they played in just 1 game, we can delete these rows
players = players[players['FG%'].notna()]

In [149]:
players[players['2P%'].isnull()]

,Player,Pos,Age,Height,Weight,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS
198,Jacob Evans,SG,22,198,95,MIN,2,0,2.0,0.0,0.5,0.000,0.0,0.5,0.000,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299,William Howard,PF,26,203,94,HOU,2,0,6.5,0.0,2.0,0.000,0.0,2.0,0.000,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,1.0,0.5,0.0,0.0,0.5,0.0
596,Jarred Vanderbilt,PF,20,206,97,MIN,2,0,2.5,0.0,0.5,0.000,0.0,0.5,0.000,0.0,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.5,0.0,0.0,0.0,0.0,1.0
613,Derrick Walton,PG,24,183,85,DET,3,0,8.7,0.3,1.0,0.333,0.3,1.0,0.333,0.0,0.0,NaN,0.5,0.0,0.0,NaN,0.0,0.3,1.7,1.0,0.0,0.3,1.0


In [150]:
#Not relevant players also, we can delete these rows
players = players[players['2P%'].notna()]

In [151]:
players[players['FT%'].isnull()]

,Player,Pos,Age,Height,Weight,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS
3,Kyle Alexander,C,23,211,99,MIA,2,0,6.5,0.5,1.0,0.500,0.0,0.0,NaN,0.5,1.0,0.500,0.500,0.0,0.0,NaN,1.0,1.5,0.0,0.0,0.0,0.5,1.0
11,Ryan Anderson,C,31,208,109,HOU,2,0,7.0,1.0,3.5,0.286,0.5,2.5,0.200,0.5,1.0,0.500,0.357,0.0,0.0,NaN,0.0,3.5,1.0,0.5,0.0,0.5,2.5
66,Jordan Bone,PG,22,191,82,DET,10,0,5.3,0.5,2.0,0.250,0.2,1.0,0.200,0.3,1.0,0.300,0.300,0.0,0.0,NaN,0.0,0.4,0.8,0.1,0.0,0.2,1.2
70,Brian Bowen,SF,21,198,86,IND,6,1,5.2,0.5,1.7,0.300,0.0,0.3,0.000,0.5,1.3,0.375,0.300,0.0,0.0,NaN,0.2,1.2,0.0,0.0,0.2,0.2,1.0
103,Devontae Cacok,C,23,201,108,LAL,1,0,9.0,3.0,6.0,0.500,0.0,0.0,NaN,3.0,6.0,0.500,0.500,0.0,0.0,NaN,2.0,5.0,1.0,0.0,0.0,0.0,6.0
121,Zylan Cheatham,PF,24,196,100,NOP,4,0,12.8,1.5,2.3,0.667,0.0,0.3,0.000,1.5,2.0,0.750,0.667,0.0,0.0,NaN,0.8,2.3,0.8,0.3,0.3,1.0,3.0
123,Chris Chiozza,PG,24,183,79,WAS,10,0,12.3,1.0,3.4,0.294,0.7,1.6,0.438,0.3,1.8,0.167,0.397,0.0,0.0,NaN,0.0,1.5,2.8,1.0,0.2,0.9,2.7
152,Jamal Crawford,SG,39,196,91,BRK,1,0,6.0,2.0,4.0,0.500,1.0,2.0,0.500,1.0,2.0,0.500,0.625,0.0,0.0,NaN,0.0,0.0,3.0,0.0,0.0,0.0,5.0
161,Troy Daniels,SG,28,193,93,DEN,6,0,12.7,1.7,4.7,0.357,1.0,3.3,0.300,0.7,1.3,0.500,0.464,0.0,0.0,NaN,0.0,1.0,0.5,0.5,0.0,0.5,4.3
189,Henry Ellenson,C,23,211,111,BRK,5,0,3.0,0.2,1.4,0.143,0.0,0.8,0.000,0.2,0.6,0.333,0.143,0.0,0.0,NaN,0.4,1.2,0.2,0.0,0.0,0.0,0.4


In [152]:
#Not relevant players (max played 10 games), we can delete these rows
players = players[players['FT%'].notna()]

In [153]:
players[players['3P%'].isnull()]

,Player,Pos,Age,Height,Weight,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS
13,Kostas Antetokounmpo,PF,22,208,91,LAL,5,0,4.0,0.6,0.6,1.000,0.0,0.0,NaN,0.6,0.6,1.000,1.000,0.2,0.4,0.500,0.4,0.6,0.4,0.0,0.0,0.2,1.4
56,Bismack Biyombo,C,27,206,111,CHO,53,29,19.4,2.8,5.2,0.543,0.0,0.0,NaN,2.8,5.2,0.543,0.543,1.7,2.8,0.603,2.3,5.8,0.9,0.2,0.9,0.9,7.4
86,Moses Brown,C,20,218,111,POR,9,0,3.7,0.4,1.1,0.400,0.0,0.0,NaN,0.4,1.1,0.400,0.400,0.3,0.9,0.375,0.3,1.6,0.1,0.1,0.1,0.3,1.2
106,Clint Capela,C,25,208,109,HOU,39,39,32.8,6.3,9.9,0.629,0.0,0.0,NaN,6.3,9.9,0.629,0.629,1.4,2.7,0.529,4.3,13.8,1.2,0.8,1.8,1.6,13.9
116,Willie Cauley-Stein,C,26,213,109,GSW,41,37,22.9,3.5,6.3,0.560,0.0,0.0,NaN,3.5,6.3,0.560,0.560,0.9,1.4,0.614,1.7,6.2,1.5,1.1,1.2,0.9,7.9
118,Tyson Chandler,C,37,216,107,HOU,26,5,8.4,0.5,0.7,0.778,0.0,0.0,NaN,0.5,0.7,0.778,0.778,0.2,0.5,0.462,1.0,2.5,0.2,0.2,0.3,0.3,1.3
142,Tyler Cook,PF-SF,22,203,116,TOT,13,0,4.2,0.6,0.9,0.667,0.0,0.0,NaN,0.6,0.9,0.667,0.667,0.5,0.6,0.875,0.5,1.1,0.1,0.2,0.0,0.2,1.8
143,Tyler Cook,PF,22,203,116,CLE,11,0,3.2,0.6,0.9,0.700,0.0,0.0,NaN,0.6,0.9,0.700,0.700,0.5,0.5,0.833,0.4,0.9,0.1,0.1,0.0,0.1,1.7
144,Tyler Cook,SF,22,203,116,DEN,2,0,9.5,0.5,1.0,0.500,0.0,0.0,NaN,0.5,1.0,0.500,0.500,1.0,1.0,1.000,1.0,2.0,0.0,1.0,0.0,1.0,2.0
163,Ed Davis,C,30,208,98,UTA,28,1,10.8,0.8,1.6,0.478,0.0,0.0,NaN,0.8,1.6,0.478,0.478,0.3,0.5,0.500,1.3,3.8,0.4,0.4,0.3,0.5,1.8


In [154]:
# Some players are relevant for a team, so instead of deleting these rows I will fill the missing values with 0
players['3P%'] = players['3P%'].fillna(0)

In [155]:
players.name = 'Players'
teams.name = 'Teams'
dfs = [players, teams]
for df in dfs:
    print('{}'.format(df.name))
    display_missing(df)

Players
Player column missing values: 0
Pos column missing values: 0
Age column missing values: 0
Height column missing values: 0
Weight column missing values: 0
Tm column missing values: 0
G column missing values: 0
GS column missing values: 0
MP column missing values: 0
FG column missing values: 0
FGA column missing values: 0
FG% column missing values: 0
3P column missing values: 0
3PA column missing values: 0
3P% column missing values: 0
2P column missing values: 0
2PA column missing values: 0
2P% column missing values: 0
eFG% column missing values: 0
FT column missing values: 0
FTA column missing values: 0
FT% column missing values: 0
ORB column missing values: 0
TRB column missing values: 0
AST column missing values: 0
STL column missing values: 0
BLK column missing values: 0
TOV column missing values: 0
PTS column missing values: 0


Teams
Team column missing values: 0
Short_team column missing values: 0
EAST/WEST column missing values: 0
PLAYOFF column missing values: 0
PLAYOFF_

## FEATURE ENGINEERING

### Leaving only relevant players for a team

In [156]:
# We're using 'or' operator because some players get traded during season and hasn't played a lot of games, even though they are relevant
important_players = players[(players['MP'] > 24) | (players['GS'] > 36)]
important_players

,Player,Pos,Age,Height,Weight,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS
0,Steven Adams,C,26,213,116,OKC,63,63,26.7,4.5,7.6,0.592,0.0,0.0,0.333,4.5,7.5,0.594,0.593,1.9,3.2,0.582,3.3,9.3,2.3,0.8,1.1,1.5,10.9
1,Bam Adebayo,PF,22,208,116,MIA,72,72,33.6,6.1,11.0,0.557,0.0,0.2,0.143,6.1,10.8,0.564,0.558,3.7,5.3,0.691,2.4,10.2,5.1,1.1,1.3,2.8,15.9
2,LaMarcus Aldridge,C,34,211,111,SAS,53,53,33.1,7.4,15.0,0.493,1.2,3.0,0.389,6.2,12.0,0.519,0.532,3.0,3.6,0.827,1.9,7.4,2.4,0.7,1.6,1.4,18.9
6,Jarrett Allen,C,21,211,106,BRK,70,64,26.5,4.3,6.6,0.649,0.0,0.1,0.000,4.3,6.6,0.658,0.649,2.4,3.9,0.633,3.1,9.6,1.6,0.6,1.3,1.1,11.1
12,Giannis Antetokounmpo,PF,25,206,93,MIL,63,63,30.4,10.9,19.7,0.553,1.4,4.7,0.304,9.5,15.0,0.631,0.589,6.3,10.0,0.633,2.2,13.6,5.6,1.0,1.0,3.7,29.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,Justise Winslow,SG,23,201,102,MIA,11,5,32.0,4.5,11.7,0.388,0.5,2.5,0.222,4.0,9.3,0.431,0.411,1.6,2.5,0.667,1.5,6.6,4.0,0.6,0.5,2.2,11.3
645,Thaddeus Young,PF,31,203,100,CHI,64,16,24.9,4.2,9.4,0.448,1.2,3.5,0.356,3.0,5.9,0.501,0.513,0.7,1.1,0.583,1.5,4.9,1.8,1.4,0.4,1.6,10.3
646,Trae Young,PG,21,188,82,ATL,60,60,35.3,9.1,20.8,0.437,3.4,9.5,0.361,5.7,11.4,0.501,0.519,8.0,9.3,0.860,0.5,4.3,9.3,1.1,0.1,4.8,29.6
647,Cody Zeller,C,27,213,109,CHO,58,39,23.1,4.3,8.3,0.524,0.3,1.3,0.240,4.0,7.0,0.577,0.543,2.1,3.1,0.682,2.8,7.1,1.5,0.7,0.4,1.3,11.1


230 players by 30 teams = 7.66 important players, 5 is in a basic team roaster + 2-3 important players -> seems it's good amount to analise

### AST / TOV

In [157]:
important_players['AST/TOV'] = (important_players['AST']/important_players['TOV'])

<ipython-input-157-d18c84f01dac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_players['AST/TOV'] = (important_players['AST']/important_players['TOV'])


In [158]:
important_players

,Player,Pos,Age,Height,Weight,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS,AST/TOV
0,Steven Adams,C,26,213,116,OKC,63,63,26.7,4.5,7.6,0.592,0.0,0.0,0.333,4.5,7.5,0.594,0.593,1.9,3.2,0.582,3.3,9.3,2.3,0.8,1.1,1.5,10.9,1.533333
1,Bam Adebayo,PF,22,208,116,MIA,72,72,33.6,6.1,11.0,0.557,0.0,0.2,0.143,6.1,10.8,0.564,0.558,3.7,5.3,0.691,2.4,10.2,5.1,1.1,1.3,2.8,15.9,1.821429
2,LaMarcus Aldridge,C,34,211,111,SAS,53,53,33.1,7.4,15.0,0.493,1.2,3.0,0.389,6.2,12.0,0.519,0.532,3.0,3.6,0.827,1.9,7.4,2.4,0.7,1.6,1.4,18.9,1.714286
6,Jarrett Allen,C,21,211,106,BRK,70,64,26.5,4.3,6.6,0.649,0.0,0.1,0.000,4.3,6.6,0.658,0.649,2.4,3.9,0.633,3.1,9.6,1.6,0.6,1.3,1.1,11.1,1.454545
12,Giannis Antetokounmpo,PF,25,206,93,MIL,63,63,30.4,10.9,19.7,0.553,1.4,4.7,0.304,9.5,15.0,0.631,0.589,6.3,10.0,0.633,2.2,13.6,5.6,1.0,1.0,3.7,29.5,1.513514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,Justise Winslow,SG,23,201,102,MIA,11,5,32.0,4.5,11.7,0.388,0.5,2.5,0.222,4.0,9.3,0.431,0.411,1.6,2.5,0.667,1.5,6.6,4.0,0.6,0.5,2.2,11.3,1.818182
645,Thaddeus Young,PF,31,203,100,CHI,64,16,24.9,4.2,9.4,0.448,1.2,3.5,0.356,3.0,5.9,0.501,0.513,0.7,1.1,0.583,1.5,4.9,1.8,1.4,0.4,1.6,10.3,1.125000
646,Trae Young,PG,21,188,82,ATL,60,60,35.3,9.1,20.8,0.437,3.4,9.5,0.361,5.7,11.4,0.501,0.519,8.0,9.3,0.860,0.5,4.3,9.3,1.1,0.1,4.8,29.6,1.937500
647,Cody Zeller,C,27,213,109,CHO,58,39,23.1,4.3,8.3,0.524,0.3,1.3,0.240,4.0,7.0,0.577,0.543,2.1,3.1,0.682,2.8,7.1,1.5,0.7,0.4,1.3,11.1,1.153846


## Calculating the number of specific player in a team - big players

In [159]:
big_passers = important_players[(important_players['Height'] > 198) & (important_players['AST'] > 4) & (important_players['AST/TOV'] > 2)]
big_shooters = important_players[(important_players['Height'] > 198) & (important_players['3PA'] > 6) & (important_players['3P%'] > 0.37)]
big_offensive = important_players[((important_players['Height'] > 198) & (important_players['PTS'] > 20)) & ((important_players['eFG%'] > 0.535) | (important_players['FG%'] > 0.5))]

In [160]:
big_shooters_passers = important_players[(important_players['Height'] > 198) & (important_players['3PA'] > 4) & (important_players['3P%'] > 0.35) & (important_players['AST'] > 3) & (important_players['AST/TOV'] > 2)]
big_defense_passers = important_players[(important_players['Height'] > 198) & (important_players['STL'] > 0.8) & (important_players['BLK'] > 0.5) & (important_players['AST'] > 3) & (important_players['AST/TOV'] > 2)]
big_defense_shooters = important_players[(important_players['Height'] > 198) & (important_players['STL'] > 0.5) & (important_players['BLK'] > 1) & (important_players['3P'] > 2) & (important_players['3P%'] > 0.3)]
big_few_tov = important_players[(important_players['Height'] > 198) & (important_players['TOV'] < 0.9)]
big_many_FT = important_players[(important_players['Height'] > 198) & (important_players['FTA'] > 5) & (important_players['FT%'] > 0.8)]

### Number of results

In [161]:
len(big_passers.index)

10

In [162]:
len(big_defense_shooters.index)

5

We're making conditions to get 10-15 players for each more extensive type and 5-10 players for more specific type.
There're 30 teamt in the NBA, so a smaller number of players for a type will allow us a more accurate check of the usability of each of them.

### Doing the same for small players

In [163]:
small_passers = important_players[(important_players['Height'] <= 198) & (important_players['AST'] > 6) &  (important_players['AST/TOV'] > 2.5)]
small_shooters = important_players[(important_players['Height'] <= 198) & (important_players['3PA'] > 7) & (important_players['3P%'] > 0.37)]
small_offensive = important_players[((important_players['Height'] <= 198) & (important_players['PTS'] > 20)) & ((important_players['eFG%'] > 0.53) | (important_players['FG%'] > 0.45))]
small_shooters_passers = important_players[(important_players['Height'] <= 198) & (important_players['3PA'] > 6) & (important_players['3P%'] > 0.37) & (important_players['AST'] > 5) & (important_players['AST/TOV'] > 2)]

In [164]:
small_few_tov = important_players[(important_players['Height'] <= 198) & (important_players['TOV'] < 1)]
small_many_FT = important_players[(important_players['Height'] <= 198) & (important_players['FTA'] > 5) & (important_players['FT%'] > 0.8)]

In [165]:
great_rebounders = important_players[(important_players['TRB'] > 8) & (important_players['ORB'] > 3)]

### Adding values to teams dataframe

In [166]:
teams_short_name = teams['Short_team'].unique()
different_players = [big_passers, big_shooters, big_offensive, big_shooters_passers, big_defense_passers, big_defense_shooters, big_few_tov, big_many_FT,
                     small_passers, small_shooters, small_offensive, small_shooters_passers, small_few_tov, small_many_FT, great_rebounders]


big_passers.name = 'big_passers'
big_shooters.name = 'big_shooters'
big_offensive.name = 'big_offensive'
big_shooters_passers.name = 'big_shooters_passers'
big_defense_passers.name = 'big_defense_passers'
big_defense_shooters.name = 'big_defense_shooters'
big_few_tov.name = 'big_few_tov'
big_many_FT.name = 'big_many_FT'

small_passers.name = 'small_passers'
small_shooters.name = 'small_shooters'
small_offensive.name = 'small_offensive'
small_shooters_passers.name = 'small_shooters_passers'
small_few_tov.name = 'small_few_tov'
small_many_FT.name = 'small_many_FT'

great_rebounders.name = 'great_rebounders'

#print('{} column missing values: {}'.format(col, df[col].isnull().sum()))


for n in different_players:
    print('Number of {}: {}'.format(n.name, len(n.index)))

Number of big_passers: 10
Number of big_shooters: 11
Number of big_offensive: 10
Number of big_shooters_passers: 4
Number of big_defense_passers: 5
Number of big_defense_shooters: 5
Number of big_few_tov: 11
Number of big_many_FT: 6
Number of small_passers: 9
Number of small_shooters: 12
Number of small_offensive: 11
Number of small_shooters_passers: 5
Number of small_few_tov: 11
Number of small_many_FT: 11
Number of great_rebounders: 11


Numbers of players for each type meets the requirements.

In [167]:
def add_new_type(df_to, df_from, team_list):
    for n in team_list:
        df_to.loc[df_to.index[df_to['Short_team'] == n], df_from.name] = df_from[df_from['Tm'] == n].count()['Player']
        
for i in different_players:
    add_new_type(teams, i, teams_short_name)

In [168]:
teams

,Team,Short_team,EAST/WEST,PLAYOFF,PLAYOFF_SCORE,big_passers,big_shooters,big_offensive,big_shooters_passers,big_defense_passers,big_defense_shooters,big_few_tov,big_many_FT,small_passers,small_shooters,small_offensive,small_shooters_passers,small_few_tov,small_many_FT,great_rebounders
0,Dallas Mavericks,DAL,EAST,yes,first round,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Milwaukee Bucks,MIL,EAST,yes,quarter-finals,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Portland Trail Blazers,POR,WEST,yes,first round,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,0.0,1.0,1.0
3,Houston Rockets,HOU,WEST,yes,quarter-finals,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0
4,Los Angeles Clippers,LAC,WEST,yes,quarter-finals,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5,New Orleans Pelicans,NOP,WEST,no,off-playoffs,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
6,Phoenix Suns,PHO,WEST,no,off-playoffs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
7,Washington Wizards,WAS,EAST,no,off-playoffs,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,Memphis Grizzlies,MEM,WEST,no,off-playoffs,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Boston Celtics,BOS,EAST,yes,semi-finals,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Mapping values

In [169]:
teams['PLAYOFF'] = teams['PLAYOFF'].map({'yes': 1, 'no': 0})

In [170]:
teams['EAST/WEST'] = teams['EAST/WEST'].map({'EAST': 1, 'WEST': 0})

In [171]:
teams['PLAYOFF_SCORE'].unique()

array(['first round', 'quarter-finals', 'off-playoffs', 'semi-finals',
       'finals', 'champions'], dtype=object)

In [172]:
teams['PLAYOFF_SCORE'] = teams['PLAYOFF_SCORE'].map({'off-playoffs':0, 'first round': 1, 'quarter-finals':2, 'semi-finals':3,
       'finals':4, 'champions':5})

## Correlations

In [174]:
df_corr = teams.drop(['PLAYOFF', 'PLAYOFF_SCORE'], axis=1).corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
df_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
df_corr.drop(df_corr.iloc[1::2].index, inplace=True)
df_train_corr_nd = df_corr.drop(df_corr[df_corr['Correlation Coefficient'] == 1.0].index)

In [175]:
# Training set high correlations
corr = df_train_corr_nd['Correlation Coefficient'] > 0.2
df_train_corr_nd[corr]

,Feature 1,Feature 2,Correlation Coefficient
16,small_few_tov,big_offensive,0.545342
18,great_rebounders,small_offensive,0.506793
20,big_defense_passers,big_passers,0.442719
22,small_offensive,big_passers,0.428825
24,big_few_tov,small_offensive,0.428825
26,small_passers,small_shooters_passers,0.428701
28,big_many_FT,small_few_tov,0.426798
30,small_passers,big_passers,0.426401
32,small_offensive,small_shooters_passers,0.410865
34,small_offensive,small_shooters,0.361702


In [179]:
df_all_corr = teams.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
df_all_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
df_all_corr[df_all_corr['Feature 1'] == 'PLAYOFF']

,Feature 1,Feature 2,Correlation Coefficient
15,PLAYOFF,PLAYOFF,1.000000
18,PLAYOFF,PLAYOFF_SCORE,0.741587
96,PLAYOFF,small_few_tov,0.260723
113,PLAYOFF,big_defense_passers,0.239046
118,PLAYOFF,big_few_tov,0.236228
120,PLAYOFF,big_passers,0.236228
155,PLAYOFF,big_offensive,0.186754
167,PLAYOFF,big_shooters_passers,0.170346
169,PLAYOFF,small_shooters_passers,0.170346
197,PLAYOFF,big_many_FT,0.133631


## Logistic regression model - is team going to qualify to the playoffs?
## V.1

#### Declaring the dependent and the indepentent variables

In [180]:
teams.columns.unique()

Index(['Team', 'Short_team', 'EAST/WEST', 'PLAYOFF', 'PLAYOFF_SCORE',
       'big_passers', 'big_shooters', 'big_offensive', 'big_shooters_passers',
       'big_defense_passers', 'big_defense_shooters', 'big_few_tov',
       'big_many_FT', 'small_passers', 'small_shooters', 'small_offensive',
       'small_shooters_passers', 'small_few_tov', 'small_many_FT',
       'great_rebounders'],
      dtype='object')

In [184]:
y = teams['PLAYOFF']
x1 = teams[['EAST/WEST', 'big_passers', 'big_shooters', 'big_offensive', 'big_shooters_passers',
       'big_defense_passers', 'big_defense_shooters', 'big_few_tov',
       'big_many_FT', 'small_passers', 'small_shooters', 'small_offensive',
       'small_shooters_passers', 'small_few_tov', 'small_many_FT',
       'great_rebounders']]

In [185]:
x1

,EAST/WEST,big_passers,big_shooters,big_offensive,big_shooters_passers,big_defense_passers,big_defense_shooters,big_few_tov,big_many_FT,small_passers,small_shooters,small_offensive,small_shooters_passers,small_few_tov,small_many_FT,great_rebounders
0,1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,0.0,1.0,1.0
3,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0
4,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
7,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


#### Regression

In [186]:
x = sm.add_constant(x1)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()
results_log.summary()

         Current function value: 0.297949
         Iterations: 35


C:\Users\Nikodem\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                PLAYOFF   No. Observations:                   30
Model:                          Logit   Df Residuals:                       13
Method:                           MLE   Df Model:                           16
Date:                Sun, 28 Feb 2021   Pseudo R-squ.:                  0.5688
Time:                        16:37:50   Log-Likelihood:                -8.9385
converged:                      False   LL-Null:                       -20.728
Covariance Type:            nonrobust   LLR p-value:                   0.09912
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -28.6526   3029.607     -0.009      0.992   -5966.573    5909.268
EAST/WEST                  3.2712      6.249      0.523      0.601      -8.977      15.520
big_passers                0.9945      6.032      0.165      0.869     -10.829      12.817
big_shooters              -1.2393      3.529     -0.351      0.725      -8.155       5.677
big_offensive              4.6535      3.796      1.226      0.220      -2.787      12.094
big_shooters_passers      18.9607   3029.610      0.006      0.995   -5918.965    5956.887
big_defense_passers       27.5481   3029.611      0.009      0.993   -5910.379    5965.476
big_defense_shooters      23.1546   3029.606      0.008      0.994   -5914.764    5961.073
big_few_tov               22.5364   3029.607      0.007      0.994   -5915.385    5960.458
big_many_FT              -18.9495   3029.602     -0.006      0.995   -5956.860    5918.961
small_passers              5.4884      4.805      1.142      0.253      -3.929      14.906
small_shooters           -20.0384   3029.601     -0.007      0.995   -5957.946    5917.870
small_offensive           15.3614   3029.611      0.005      0.996   -5922.566    5953.289
small_shooters_passers    30.3688   3029.641      0.010      0.992   -5907.619    5968.356
small_few_tov             19.4101   3029.600      0.006      0.995   -5918.498    5957.318
small_many_FT              1.4488      3.335      0.434      0.664      -5.088       7.986
great_rebounders           1.9061      4.114      0.463      0.643      -6.156       9.969
==========================================================================================

Possibly complete quasi-separation: A fraction 0.37 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

### Accuracy

In [187]:
cm_df = pd.DataFrame(results_log.pred_table())
cm_df.columns = ['Predicted 0','Predicted 1']
cm_df = cm_df.rename(index={0: 'Actual 0',1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,12.0,2.0
Actual 1,3.0,13.0


In [188]:
cm = np.array(cm_df)
accuracy_train = (cm[0,0]+cm[1,1])/cm.sum()
accuracy_train

0.8333333333333334

## 2nd version

#### Many features in our model are not statistically significant. P>[z] values near 1. We want them below 0.05.
#### Also maximum number of iterations has been exceeded

In [199]:
x2 = teams[['big_passers', 'big_shooters', 'big_offensive', 'big_few_tov',
       'big_many_FT', 'small_shooters', 'small_offensive',
       'small_shooters_passers']]

In [200]:
x = sm.add_constant(x2)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()
results_log.summary()

Optimization terminated successfully.
         Current function value: 0.539241
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                PLAYOFF   No. Observations:                   30
Model:                          Logit   Df Residuals:                       21
Method:                           MLE   Df Model:                            8
Date:                Sun, 28 Feb 2021   Pseudo R-squ.:                  0.2195
Time:                        17:12:52   Log-Likelihood:                -16.177
converged:                       True   LL-Null:                       -20.728
Covariance Type:            nonrobust   LLR p-value:                    0.3339
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0.7212      0.871     -0.828      0.408      -2.428       0.986
big_passers                2.0505      1.250      1.640      0.101      -0.399       4.500
big_shooters              -1.1084      1.072     -1.034      0.301      -3.209       0.992
big_offensive              0.9219      0.864      1.067      0.286      -0.771       2.615
big_few_tov                1.9516      1.292      1.511      0.131      -0.580       4.484
big_many_FT               -1.5430      1.520     -1.015      0.310      -4.522       1.436
small_shooters            -1.7039      1.192     -1.430      0.153      -4.039       0.631
small_offensive            0.6556      0.962      0.682      0.495      -1.229       2.540
small_shooters_passers     3.1874      2.059      1.548      0.122      -0.849       7.224
==========================================================================================
"""

In [201]:
cm_df = pd.DataFrame(results_log.pred_table())
cm_df.columns = ['Predicted 0','Predicted 1']
cm_df = cm_df.rename(index={0: 'Actual 0',1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,10.0,4.0
Actual 1,4.0,12.0


In [202]:
cm = np.array(cm_df)
accuracy_train = (cm[0,0]+cm[1,1])/cm.sum()
accuracy_train

0.7333333333333333

### 3rd version - We're trying to get higher accuracy score with reduced number of features statistically insignificant features

In [460]:
x3 = teams[['EAST/WEST', 'big_passers', 'big_offensive', 'big_shooters_passers',
       'big_defense_passers', 'big_defense_shooters', 'big_few_tov',
       'big_many_FT', 'small_passers', 'small_shooters',
       'small_shooters_passers', 'small_few_tov', 'small_many_FT']]

In [461]:
x = sm.add_constant(x3)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()
results_log.summary()

Optimization terminated successfully.
         Current function value: 0.360345
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                PLAYOFF   No. Observations:                   30
Model:                          Logit   Df Residuals:                       16
Method:                           MLE   Df Model:                           13
Date:                Sun, 28 Feb 2021   Pseudo R-squ.:                  0.4785
Time:                        20:26:38   Log-Likelihood:                -10.810
converged:                       True   LL-Null:                       -20.728
Covariance Type:            nonrobust   LLR p-value:                   0.09941
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -10.5156      6.568     -1.601      0.109     -23.389       2.358
EAST/WEST                  2.1521      2.064      1.043      0.297      -1.893       6.197
big_passers                1.2067      2.547      0.474      0.636      -3.784       6.198
big_offensive              2.6224      1.926      1.361      0.173      -1.153       6.398
big_shooters_passers       4.1930      4.150      1.010      0.312      -3.941      12.327
big_defense_passers        9.5345      6.339      1.504      0.133      -2.890      21.959
big_defense_shooters       8.7641      6.034      1.452      0.146      -3.063      20.591
big_few_tov                6.1138      4.423      1.382      0.167      -2.556      14.783
big_many_FT               -5.2062      3.605     -1.444      0.149     -12.272       1.860
small_passers              3.5779      2.546      1.405      0.160      -1.413       8.568
small_shooters            -5.1722      3.224     -1.604      0.109     -11.492       1.147
small_shooters_passers    15.4277      9.805      1.573      0.116      -3.790      34.645
small_few_tov              4.8320      3.792      1.274      0.203      -2.601      12.265
small_many_FT              2.1485      1.780      1.207      0.227      -1.340       5.637
==========================================================================================

Possibly complete quasi-separation: A fraction 0.17 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [462]:
cm_df = pd.DataFrame(results_log.pred_table())
cm_df.columns = ['Predicted 0','Predicted 1']
cm_df = cm_df.rename(index={0: 'Actual 0',1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,11.0,3.0
Actual 1,1.0,15.0


In [463]:
cm = np.array(cm_df)
accuracy_train = (cm[0,0]+cm[1,1])/cm.sum()
accuracy_train

0.8666666666666667

## Summary 

Based on the coefficion table we can assume that the biggest positive impact for team trying to get to the playoffs have: small_shooters_passers (15.43), big_defense_passers(9.53) and big_defense_shooters(8.76).

In [464]:
np.exp(15.43)

5025321.550442785

In [450]:
np.exp(9.53)

13766.591084005511

In [451]:
np.exp(8.76)

6374.11157799139

Also we have some features with negative impract: big_many_FT(-5.21) and small_shooters(-5.17).